In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\softwares\conda4.8.2-python3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\softwares\conda4.

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torch.optim import Adam

In [3]:
import os.path as osp

In [4]:
from tqdm import tqdm, tqdm_notebook

In [5]:
from pytorch_transformers import *

In [6]:
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import os
from pathlib import Path
import os.path as osp

Using TensorFlow backend.


In [7]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 17px;
</style>

In [8]:
data_path = osp.join(Path(os.getcwd()).parent,'Data')
dj = pd.read_csv(data_path  + "/stock_original.csv")
news = pd.read_csv(data_path + "/news_modified.csv")

In [9]:
dj.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Volume       float64
Adj Close    float64
cap          float64
dtype: object

In [10]:
dj['Date'] = pd.to_datetime(dj['Date']).dt.strftime('%d-%m-%Y')

In [11]:
dj

,Date,Open,High,Low,Close,Volume,Adj Close,cap
0,08-08-2008,11432.089844,11759.959961,11388.040039,11734.320312,212830000.0,11734.320312,297.580078
1,11-08-2008,11729.669922,11867.110352,11675.530273,11782.349609,183190000.0,11782.349609,52.030273
2,12-08-2008,11781.700195,11782.349609,11601.519531,11642.469727,173590000.0,11642.469727,-148.890625
3,13-08-2008,11632.809570,11633.780273,11453.339844,11532.959961,182550000.0,11532.959961,-100.739258
4,14-08-2008,11532.070312,11718.280273,11450.889648,11615.929688,159790000.0,11615.929688,79.139649
...,...,...,...,...,...,...,...,...
3031,24-08-2020,28077.580078,28314.939453,28041.750000,28308.460938,383880000.0,28308.460938,269.839844
3032,25-08-2020,28347.419922,28400.740234,28094.570313,28248.439453,338420000.0,28248.439453,-89.539063
3033,26-08-2020,28257.880859,28353.800781,28153.910156,28331.919922,327070000.0,28331.919922,126.189454
3034,27-08-2020,28384.070313,28634.220703,28363.929688,28492.269531,404340000.0,28492.269531,217.218750


In [12]:
news['Date'] = pd.to_datetime(news['Date'])

In [13]:
dj.isnull().sum() #No missing data
news.isnull().sum() #No missing data

# Compare the number of unique dates. We want matching values.
print(len(set(dj.Date)))
print(len(set(news.Date)))

# Remove the extra dates that are in news
news = news[news.Date.isin(dj.Date)]
print(len(set(dj.Date)))
print(len(set(news.Date)))

# Remove unwanted features - keep the 'Open' price only
dj = dj.drop(['High','Low','Close','Volume','Adj Close'], 1)


# Calculate the difference in opening prices between the following and current day.
# The model will try to predict the change in Open value based on the today's news.
dj = dj.set_index('Date')

# Target variable = Tomorrow's Open Price - Today's Open Price
dj = -1 * dj.diff(periods=1)

dj['Date'] = dj.index
dj = dj.reset_index(drop=True)

dj = dj[dj.Open.notnull()]


3036
3847
3036
2597


In [14]:
def decontracted(phrase):
    if "'" in phrase:
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [15]:
def clean_text(text):
    '''Remove unwanted characters and format the text to create fewer nulls word embeddings'''    
    # Convert words to lower case
    text = text.lower()    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        # Remove the contractions
        for word in text:
            new_text.append(decontracted(word))
        # Recreate the sentence
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'0,0', '00', text) 
    text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\$', ' $ ', text)
    text = re.sub(r'u s ', ' united states ', text)
    text = re.sub(r'u n ', ' united nations ', text)
    text = re.sub(r'u k ', ' united kingdom ', text)
    text = re.sub(r'j k ', ' jk ', text)
    text = re.sub(r' s ', ' ', text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r' l g b t ', ' lgbt ', text)
    text = re.sub(r'0km ', '0 km ', text)
    
    # Remove stop words
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)

    return text
    

In [16]:
news.head()

,Date,News
1525,2008-08-08,b'Georgian troops retreat from S. Osettain cap...
1526,2008-08-08,"b""The 'enemy combatent' trials are nothing but..."
1527,2008-08-08,"b""Breaking: Georgia invades South Ossetia, Rus..."
1528,2008-08-08,b'150 Russian tanks have entered South Ossetia...
1529,2008-08-08,b'Did the U.S. Prep Georgia for War with Russia?'


In [17]:
#Create a list of the opening prices and their corresponding daily headlines from the news
#Define/Initialize the variables

#text preprocessing
max_headline_length = 20
max_daily_length = 400

from nltk.tokenize import word_tokenize

price = []
headlines = []
# For all the rows in the dataframe
for row in dj.iterrows():
    # define a new variable to store all the headlines for the day
    daily_headlines = []
    # Spot the date in the given row
    date = row[1]['Date']
    # Store the price for the date
    price.append(row[1]['Open'])
    for row_ in news[news.Date==date].iterrows():
        text = clean_text(row_[1]['News'])
        text_arr = word_tokenize(text)
        text_arr = text_arr[:max_headline_length]
        text_str = " ".join(text_arr)
        daily_headlines.append(text_str)
        
    # Append the headlines for the date
    headlines.append(daily_headlines)
    # Track progress
    

In [18]:
#Join all the news headlines in a day to to form a sentence
all_headlines = []
all_headline_tokens = []
for head_line in headlines:
    daily_news = ' '
    for daily_headline in head_line:
        daily_news = daily_news + daily_headline + ' '        
    all_headlines.append(daily_news)

In [19]:
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in all_headlines]

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [21]:
# Normalize opening prices (target values)
max_price = max(price)
min_price = min(price)
mean_price = np.mean(price)
def normalize(price):
    return ((price-min_price)/(max_price-min_price))

norm_price = []
for p in price:
    norm_price.append(normalize(p))

In [22]:
##uncomment all three lines below to convert to ids
input_ids=[]
for i in tqdm_notebook(range(len(tokenized_texts))):
       input_ids.append(tokenizer.convert_tokens_to_ids(tokenized_texts[i]))

C:\softwares\conda4.8.2-python3.7\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (745 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (760 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (727 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (758 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (756 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (555 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (702 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (738 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (661 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [23]:
len(input_ids[1])

306

In [24]:
MAX_LEN = 400
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [25]:
#Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:    
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [26]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, norm_price,random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=56, test_size=0.2)

In [27]:
#Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.from_numpy(np.array(train_inputs)).long()
validation_inputs =  torch.from_numpy(np.array(validation_inputs)).long()
train_labels = torch.from_numpy(np.array(train_labels)).float()
validation_labels = torch.from_numpy(np.array(validation_labels)).float()
train_masks = torch.from_numpy(np.array(train_masks)).float()
validation_masks = torch.from_numpy(np.array(validation_masks)).float()

In [28]:
batch_size = 4

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [29]:
## Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
USE_CUDA = True
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
bert_len = 768
T_epsilon = 1e-7

In [31]:
hidden_size=128
BERT_out_dropout=0.5
BATCH_SIZE=128

In [32]:
from transformers import BertModel

In [33]:
class Caps_Layer(nn.Module):
    def __init__(self, input_dim_capsule=bert_len, num_capsule=Num_capsule, dim_capsule=Dim_capsule, \
                 routings=Routings, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Caps_Layer, self).__init__(**kwargs)

        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size 
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = self.squash
        else:
            self.activation = nn.ReLU(inplace=True)

        if self.share_weights:
            self.W = nn.Parameter(
                nn.init.xavier_normal_(torch.empty(1, input_dim_capsule, self.num_capsule * self.dim_capsule)))
        else:
            self.W = nn.Parameter(
                torch.randn(BATCH_SIZE, input_dim_capsule, self.num_capsule * self.dim_capsule))  # 64 batch_size

    def forward(self, x):

        if self.share_weights:
            u_hat_vecs = torch.matmul(x, self.W)
        else:
            print('add later')

        batch_size = x.size(0)
        input_num_capsule = x.size(1)
        u_hat_vecs = u_hat_vecs.view((batch_size, input_num_capsule,self.num_capsule, self.dim_capsule))                                     
        u_hat_vecs = u_hat_vecs.permute(0, 2, 1, 3)  # (batch_size,num_capsule,input_num_capsule,dim_capsule)
        b = torch.zeros_like(u_hat_vecs[:, :, :, 0])  # (batch_size,num_capsule,input_num_capsule)

        for i in range(self.routings):
            b = b.permute(0, 2, 1)
            
            c = F.softmax(b, dim=2)
            c = c.permute(0, 2, 1)
            b = b.permute(0, 2, 1)
            outputs = self.activation(torch.einsum('bij,bijk->bik', (c, u_hat_vecs)))  # batch matrix multiplication
            # outputs shape (batch_size, num_capsule, dim_capsule)
            if i < self.routings - 1:
                b = torch.einsum('bik,bijk->bij', (outputs, u_hat_vecs))  # batch matrix multiplication
        return outputs  # (batch_size, num_capsule, dim_capsule)

    # text version of squash, slight different from original one
    def squash(self, x, axis=-1):
        s_squared_norm = (x ** 2).sum(axis, keepdim=True)
        scale = torch.sqrt(s_squared_norm + T_epsilon)
        return x / scale

In [34]:
class Dense_Layer(nn.Module):
    def __init__(self):
        super(Dense_Layer, self).__init__()
        self.fc = nn.Sequential(
            nn.Dropout(p=dropout_p, inplace=True),
            nn.Linear(Num_capsule * Dim_capsule, 1) 
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        return self.fc(x)

In [35]:
class Linear(nn.Module):
    def __init__(self, fan_in, fan_out):
        super(Linear, self).__init__()

        self.linear = nn.Linear(fan_in, fan_out)

        nn.init.xavier_uniform_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)

    def forward(self, x):
        return self.linear(x)

In [36]:
class bertCapsuleModel(nn.Module):

    def __init__(self, input_dim_capsule=bert_len,num_capsule=Num_capsule, dim_capsule=Dim_capsule, 
                 routings=Routings, kernel_size=(9, 1),freeze_bert = False):
        
        super(bertCapsuleModel, self).__init__()
                                              
        self.bert = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=False)  
        self.capsule = Caps_Layer(input_dim_capsule=bert_len, num_capsule=Num_capsule, dim_capsule=Dim_capsule,
                                 routings=Routings,kernel_size=(9, 1))
        self.dense_layer = Dense_Layer()

    def forward(self, seq , attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''
        #Feeding the input to BERT model to obtain contextualized representations
              
        last_hidden_states, _, all_hidden_states = self.bert(seq, attention_mask=attn_masks)        
        caps_op = self.capsule(last_hidden_states) 
        output = self.dense_layer(caps_op)

        print('output shape ',output.shape)
        #Feeding cls_rep to the regression layer
        return output

In [37]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [38]:
## Define model
LR = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = bertCapsuleModel(freeze_bert = True)
criterion = RMSELoss()
if torch.cuda.is_available():
    model = model.to(device) 
    criterion.to(device) 
optimizer = AdamW(model.parameters(),lr=LR,correct_bias=False) 

In [39]:
model

bertCapsuleModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [40]:
# Store our loss and accuracy for plotting
train_loss_set = []

epochs = 5



# trange is a tqdm wrapper around the normal python range
for epoch in tqdm_notebook(range(epochs)): 
    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
            
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)            
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch 
      # Forward pass
      output = model.forward(b_input_ids, b_input_mask)           
      # loss
      loss = criterion(output, b_labels) 
      #append all losses for plotting
      train_loss_set.append(loss) 

      optimizer.zero_grad()           # clear gradients for this training step
      loss.backward()                 # backpropagation, compute gradients
      optimizer.step()                # apply gradients
    
      if (i) % 50 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss))                  


C:\softwares\conda4.8.2-python3.7\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


output shape  torch.Size([4, 1])


C:\softwares\conda4.8.2-python3.7\lib\site-packages\torch\nn\modules\loss.py:432: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Epoch [1/5], Loss: 0.3873
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  tor

output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
Epoch [1/5], Loss: 0.0532
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  tor

...

output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
Epoch [4/5], Loss: 0.0306
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  tor

output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
Epoch [4/5], Loss: 0.0363
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  tor

output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output sha

output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output sha

output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output sha

In [41]:
#torch.save(model.state_dict(), 'bert-capsule.ckpt')

In [42]:
# Test the model
predictions = []
test_labels = []

with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
        
        batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
        outputs = model.forward(b_input_ids,b_input_mask)
      # print (outputs)target.
        test_label = b_labels.cpu().data.numpy().tolist()
        test_labels.append(test_label)
        prediction = outputs.cpu().data.numpy().flatten().tolist()   
        predictions.append(prediction)
       

output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output shape  torch.Size([4, 1])
output sha

In [43]:
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

In [44]:
test_labels_arr = []
for x in test_labels:
    for y in x:
        test_labels_arr.append(y)

In [45]:
predictions_arr = []
for x in predictions:
    for y in x:
        predictions_arr.append(y)

In [46]:
mse(test_labels_arr, predictions_arr,squared=False)

0.050664572132103146

In [47]:
mae(test_labels_arr, predictions_arr)

0.02510964870452881